In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
from PIL import Image, ImageTk
import numpy as np
import joblib
import os

# Initialize the main window
root = tk.Tk()
root.title("GUI model for Classifying Concrete CS Based on 7 Constituents & Curing Age")

# Create the main canvas
canvas = tk.Canvas(root, width=1000, height=750, background='#FFFFFF')
canvas.pack()

# Load the image from the desktop and create an image on the canvas using PIL
image_path = "C:/Users/asus1/Desktop/image.jpg"

try:
    image = Image.open(image_path)
except FileNotFoundError:
    messagebox.showerror("Error", f"The file at {image_path} was not found. Please check the path and try again.")
    root.destroy()
    exit()

# Scale the image to fit the canvas if needed
image = image.resize((1000, 200), Image.LANCZOS)

# Store the image in an attribute of root to prevent garbage collection
root.image_storage = ImageTk.PhotoImage(image)
canvas.create_image(500, 200, image=root.image_storage, anchor="center")

# Function to create labels on the canvas
def create_label(text, font, fg, bg, x, y):
    label = tk.Label(root, text=text, font=font, fg=fg, bg=bg)
    canvas.create_window(x, y, anchor="w", window=label)
    return label

# Define model paths
model_paths = {
    "Random Forest": "C:/Users/asus1/Desktop/tuned_random_forest_model.joblib",
    "AdaBoost": "C:/Users/asus1/Desktop/tuned_adaboost_model.joblib",
    "XGBoost": "C:/Users/asus1/Desktop/tuned_xgb_model.joblib",
    "LightGBM": "C:/Users/asus1/Desktop/tuned_lightgbm_model.joblib",
    "CatBoost": "C:/Users/asus1/Desktop/tuned_catboost_model.joblib"
}

# Load the initial model (CatBoost as an example)
initial_model_path = model_paths["CatBoost"]
if not os.path.exists(initial_model_path):
    messagebox.showerror("Error", "Initial model file not found.")
    root.destroy()
    exit()

model = joblib.load(initial_model_path)

# Define the GUI elements
label_inputdefinetitle2 = create_label('GUI model for Classifying Concrete CS Based on 7 Constituents & Curing Age', ('Consolas', 18, 'bold', 'underline'), '#0000FF', '#FFFF00', 20, 30)
label_inputdefinetitle3 = create_label('Developers: Mohamed K. Elshaarawy, Abdelrahman K. Hamed & Mostafa M. Alsaadawi', ('Consolas', 14, 'bold'), '#C00000', '#FFFFFF', 20, 70)
label_inputdefinetitle = create_label('Definition of Parameters', ('Consolas', 16, 'bold'), '#000000', '#FFFFFF', 50, 310)

param_labels = [
    'X1: Cement (kg/m^3)',
    'X2: Blast Furnace Slag (kg/m^3)',
    'X3: Fly Ash (kg/m^3)',
    'X4: Water (kg/m^3)',
    'X5: Superplasticizer (kg/m^3)',
    'X6: Coarse Aggregate (kg/m^3)',
    'X7: Fine Aggregate (kg/m^3)',
    'X8: Age (day)',
]
for i, text in enumerate(param_labels, start=1):
    create_label(text, ('Consolas', 14), '#00008B', '#FFFFFF', 50, 320 + i * 30)

label_inputs = create_label('Inputs', ('Consolas', 16, 'bold'), '#000000', '#FFFFFF', 620, 310)

entry_fields = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']
entries = {}
for i, field in enumerate(entry_fields, start=1):
    create_label(f'{field} = ', ('Consolas', 14, 'bold'), '#006600', '#FFFFFF', 550, 320 + i * 30)
    entry = tk.Entry(root, font=('Consolas', 14, 'bold'), bg='#F0F0F0', highlightbackground='#000000', highlightthickness=1, bd=0, width=20)
    canvas.create_window(750, 320 + i * 30, window=entry)
    entries[field] = entry

label_output = create_label('Output:', ('Consolas', 16, 'bold'), '#000000', '#FFFFFF', 50, 610)
label_result = create_label('Concrete compressive strength class (0=Low CS, 1=Normal CS, 2=High CS)', ('Consolas', 14), '#C00000', '#FFFFFF', 180, 610)

# Dropdown menu for model selection
def load_model(event):
    selected_model = model_choice.get()
    model_path = model_paths[selected_model]
    if not os.path.exists(model_path):
        messagebox.showerror("Error", f"{selected_model} model file not found.")
        return
    global model
    model = joblib.load(model_path)

model_choice = tk.StringVar(root)
model_choice.set("CatBoost")  # default value
model_menu = ttk.Combobox(root, textvariable=model_choice, values=list(model_paths.keys()), state="readonly", font=('Consolas', 14))
model_menu.bind("<<ComboboxSelected>>", load_model)
canvas.create_window(550, 650, window=model_menu)

# Predict button command function
def predict():
    # Collect input data
    input_values = []
    for field in entry_fields:
        try:
            value = float(entries[field].get())
            if value < 0:
                raise ValueError("Value must be non-negative.")
            input_values.append(value)
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input for {field}. Please enter a non-negative numeric value.")
            return

    input_data = np.array([input_values])
    try:
        prediction = model.predict(input_data)
        label_result['text'] = f'CS Class= {int(prediction[0])} (0=Low CS, 1=Normal CS, 2=High CS)'
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")

# Predict button
predict_button = tk.Button(root, text='Calculate', font=('Consolas', 16), command=predict, fg='#FFFFFF', bg='dark blue')
canvas.create_window(750, 650, window=predict_button)

# Run the main loop
root.mainloop()
